In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
import math

In [2]:
#Reading in 2016 data we cleaned
df2016 = pd.read_csv('data/exported_data.csv')


In [3]:
df2016

,Democrats 2016,Republicans 2016,Democrats 2012,Republicans 2012,State,County,Fips,Less Than High School Diploma,At Least High School Diploma,White (Not Latino) Population,Asian American Population,African American Population,Construction.extraction.maintenance.and.repair.occupations,Average Unemployment % 2016,"Poverty Percent, All Ages",realGDP2016,At Least Bachelors's Degree,Graduate Degree,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
0,33.264033,62.176022,45.084746,53.587571,Michigan,"Presque Isle County, Michigan",26141,14.7,85.3,97.05,0.20,0.30,13.85,9.6,14.7,393475,14.3,4.7,0.070504,0.037034,0.033471,0.193137,0.097412,0.095725,0.376055,0.185824,0.190231,0.360304,0.178417,0.181886
1,41.088014,53.565475,50.177448,48.576131,Michigan,"Calhoun County, Michigan",26025,12.9,87.1,80.65,1.60,10.45,8.35,4.8,16.8,5594998,18.8,6.5,0.118989,0.061094,0.057895,0.308407,0.151431,0.156975,0.352965,0.173308,0.179657,0.219640,0.096424,0.123215
2,28.744694,66.858702,40.316090,58.523357,Michigan,"Branch County, Michigan",26023,13.4,86.6,90.35,0.60,3.05,8.05,4.8,16.8,1245185,14.1,4.8,0.101433,0.054194,0.047239,0.309689,0.170651,0.139038,0.362648,0.189468,0.173180,0.226230,0.103059,0.123171
3,36.855246,56.491599,51.061492,47.394519,Michigan,"Shiawassee County, Michigan",26155,10.7,89.3,95.45,0.30,0.45,12.05,5.3,12.0,1602095,14.9,5.6,0.110296,0.057764,0.052533,0.289581,0.146615,0.142966,0.379568,0.186526,0.193042,0.220555,0.099090,0.121465
4,34.247700,61.432326,43.482538,55.697655,Michigan,"Mackinac County, Michigan",26097,10.9,89.1,76.80,0.60,1.65,13.85,9.9,13.7,616120,20.1,7.6,0.078798,0.043072,0.035726,0.220924,0.114079,0.106845,0.384307,0.193322,0.190985,0.315971,0.155036,0.160935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,24.798206,69.058296,38.451209,58.460161,Minnesota,"Lake of the Woods County, Minnesota",27077,10.4,89.6,95.55,0.45,0.25,9.90,4.8,10.9,167599,18.6,3.8,0.083898,0.044208,0.039690,0.225234,0.112294,0.112940,0.413682,0.218135,0.195547,0.277186,0.139400,0.137786
1060,39.551402,52.336449,48.513800,48.938429,Minnesota,"Stevens County, Minnesota",27149,10.3,89.7,92.30,1.05,1.20,5.90,3.0,11.1,618864,23.5,8.4,0.261536,0.126063,0.135473,0.263857,0.143207,0.120650,0.263083,0.131993,0.131090,0.211524,0.092807,0.118716
1061,26.105645,67.456300,36.962071,60.694748,Minnesota,"Martin County, Minnesota",27091,13.1,86.9,95.00,0.75,0.30,8.55,4.1,12.1,1014414,17.5,4.9,0.094755,0.048661,0.046094,0.259678,0.132888,0.126789,0.358477,0.180394,0.178083,0.287090,0.129293,0.157797
1062,35.231714,58.626466,51.055766,46.616134,Minnesota,"Traverse County, Minnesota",27155,11.9,88.1,93.50,0.15,0.15,12.55,3.9,12.7,204156,14.7,2.7,0.094589,0.049943,0.044646,0.235717,0.122588,0.113129,0.354900,0.181990,0.172910,0.314794,0.144154,0.170639


In [4]:
# Reading in 2020 data we collected 

# Population broken down by age
state13 = pd.read_csv('data/cc-est2019-agesex-13.csv')
state19 = pd.read_csv('data/cc-est2019-agesex-19.csv')
state12 = pd.read_csv('data/cc-est2019-agesex-12.csv')
state27 = pd.read_csv('data/cc-est2019-agesex-27.csv')
state32 = pd.read_csv('data/cc-est2019-agesex-32.csv')
state37 = pd.read_csv('data/cc-est2019-agesex-37.csv')
state39 = pd.read_csv('data/cc-est2019-agesex-39.csv')
state42 = pd.read_csv('data/cc-est2019-agesex-42.csv')
state48 = pd.read_csv('data/cc-est2019-agesex-48.csv')
state55 = pd.read_csv('data/cc-est2019-agesex-55.csv')
state26 = pd.read_csv('data/cc-est2019-agesex-26.csv')
state4 = pd.read_csv('data/cc-est2019-agesex-04.csv')
df_states = state13.append(state19).append(state12).append(state27).append(state32).append(state37).append(state39).append(state42).append(state48).append(state55).append(state26).append(state4)

df_states = df_states[df_states['YEAR'] == 12] #12 represents 2019 (as close to 2020 as we could find)




In [5]:
df_states.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3


In [6]:
df_states

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,50,4,19,Arizona,Pima County,12,1047279,515596,531683,57018,29143,27875,109409,55916,53493,48880,24971,23909,121421,61475,59946,856311,417927,438384,831972,405566,426406,414104,211581,202523,256021,131400,124621,241976,116100,125876,212554,96591,115963,57018,29143,27875,59650,30372,29278,61977,31809,30168,686

In [7]:
# Getting the columns we want
df_states = df_states[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','AGE18PLUS_TOT','AGE18PLUS_MALE','AGE18PLUS_FEM', 'AGE1824_TOT', 'AGE1824_MALE','AGE1824_FEM', 
                       'AGE2544_TOT','AGE2544_MALE','AGE2544_FEM', 'AGE4564_TOT','AGE4564_MALE', 'AGE4564_FEM',
                      'AGE65PLUS_TOT', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM']]

In [8]:
# Dividing by total value to get a percentage 
df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
df_states['1824F_per'] = df_states['AGE1824_FEM']/df_states['AGE18PLUS_TOT']
df_states['2544tot_per'] = df_states['AGE2544_TOT']/df_states['AGE18PLUS_TOT']
df_states['2544M_per'] = df_states['AGE2544_MALE']/df_states['AGE18PLUS_TOT']
df_states['2544F_per'] = df_states['AGE2544_FEM']/df_states['AGE18PLUS_TOT']
df_states['4564tot_per'] = df_states['AGE4564_TOT']/df_states['AGE18PLUS_TOT']
df_states['4564M_per'] = df_states['AGE4564_MALE']/df_states['AGE18PLUS_TOT']
df_states['4564F_per'] = df_states['AGE4564_FEM']/df_states['AGE18PLUS_TOT']
df_states['65tot_per'] = df_states['AGE65PLUS_TOT']/df_states['AGE18PLUS_TOT']
df_states['65M_per'] = df_states['AGE65PLUS_MALE']/df_states['AGE18PLUS_TOT']
df_states['65F_per'] = df_states['AGE65PLUS_FEM']/df_states['AGE18PLUS_TOT']

<ipython-input-8-195acd64b013>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
<ipython-input-8-195acd64b013>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
<ipython-input-8-195acd64b013>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [9]:
df_states.head()

,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235
23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245
35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596
47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701
59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109


In [10]:
df_states['state_county_fips'] = 0
df_states = df_states.reset_index()
for i in range(0,len(df_states)):
    if(df_states['STATE'][i]) < 10:
        if (df_states['COUNTY'][i]) < 10:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
        elif (df_states['COUNTY'][i]) < 100:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '0' + str(df_states['COUNTY'][i])
        else:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '' + str(df_states['COUNTY'][i])
    elif (df_states['COUNTY'][i]) < 10:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
    elif (df_states['COUNTY'][i]) < 100:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '0' + str(df_states['COUNTY'][i])
    else:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '' + str(df_states['COUNTY'][i])
df_states.head()

<ipython-input-10-de49d781cb5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['state_county_fips'] = 0
<ipython-input-10-de49d781cb5c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
<ipython-input-10-de49d781cb5c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235,13001
1,23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245,13003
2,35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596,13005
3,47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701,13007
4,59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109,13009


In [11]:
# 2019 American Community Survey data about education level
acs_edu = pd.read_csv('data/acs_data_education.csv', header = 1)

In [12]:
acs_edu

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT

In [13]:
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT

In [14]:
# 2019 American Community Survey data about racial makeup
acs_race = pd.read_csv('data/acs_data_race.csv', header = 1)

In [15]:
acs_race

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,Estimate!!Total:!!Asian alone,Margin of Error!!Total:!!Asian alone,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,0500000US04001,"Apache County, Arizona",71887.0,*****,15109.0,899.0,721.0,434.0,53480.0,1032.0,160.0,168.0,15.0,27.0,1127.0,595.0,1275.0,743.0,637.0,509.0,638.0,514.0
1,0500000US04003,"Cochise County, Arizona",125922.0,*****,108177.0,1768.0,6163.0,1237.0,1006.0,911.0,2795.0,674.0,129.0,218.0,2141.0,1071.0,5511.0,1259.0,448.0,334.0,5063.0,1164.0
2,0500000US04005,"Coconino County, Arizona",143476.0,*****,91649.0,1736.0,1364.0,461.0,37187.0,1082.0,2730.0,481.0,419.0,440.0,3897.0,1334.0,6230.0,1766.0,253.0,305.0,5977.0,1750.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,*****,3547155.0,17870.0,266128.0,7213.0,93358.0,6179.0,192301.0,5275.0,10045.0,1308.0,213953.0,16250.0,162474.0,9681.0,31995.0,5459.0,130479.0,8908.0
4,0500000US04015,"Mohave County, Arizona",212181.0,*****,188756.0,3072.0,1496.0,845.0,4358.0,1173.0,1923.0,474.0,138.0,225.0,7608.0,3077.0,7902.0,1920.0,1169.0,605.0,6733.0,1827.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,0500000US55127,"Walworth County, Wisconsin",103868.0,*****,97838.0,1044.0,1416.0,568.0,359.0,353.0,409.0,310.0,0.0,153.0,1948.0,971.0,1898.0,658.0,317.0,244.0,1581.0,605.0
337,0500000US55131,"Washington County, Wisconsin",136034.0,*****,128012.0,751.0,2501.0,490.0,224.0,323.0,2643.0,506.0,57.0,104.0,1315.0,762.0,1282.0,798.0,395.0,434.0,887.0,645.0
338,0500000US55133,"Waukesha County, Wisconsin",404198.0,*****,370945.0,2169.0,6660.0,1419.0,912.0,363.0,15415.0,1092.0,257.0,256.0,3725.0,2262.0,6284.0,1847.0,696.0,506.0,5588.0,1773.0
339,0500000US55139,"Winnebago County, Wisconsin",171907.0,*****,155315.0,2212.0,4049.0,781.0,1084.0,343.0,4560.0,1278.0,986.0,1311.0,3186.0,2230.0,2727.0,998.0,51.0,81.0,2676.0,1016.0


In [16]:
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,Estimate!!Total:!!Asian alone,Margin of Error!!Total:!!Asian alone,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,0500000US04001,"Apache County, Arizona",71887.0,*****,15109.0,899.0,721.0,434.0,53480.0,1032.0,160.0,168.0,15.0,27.0,1127.0,595.0,1275.0,743.0,637.0,509.0,638.0,514.0
1,0500000US04003,"Cochise County, Arizona",125922.0,*****,108177.0,1768.0,6163.0,1237.0,1006.0,911.0,2795.0,674.0,129.0,218.0,2141.0,1071.0,5511.0,1259.0,448.0,334.0,5063.0,1164.0
2,0500000US04005,"Coconino County, Arizona",143476.0,*****,91649.0,1736.0,1364.0,461.0,37187.0,1082.0,2730.0,481.0,419.0,440.0,3897.0,1334.0,6230.0,1766.0,253.0,305.0,5977.0,1750.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,*****,3547155.0,17870.0,266128.0,7213.0,93358.0,6179.0,192301.0,5275.0,10045.0,1308.0,213953.0,16250.0,162474.0,9681.0,31995.0,5459.0,130479.0,8908.0
4,0500000US04015,"Mohave County, Arizona",212181.0,*****,188756.0,3072.0,1496.0,845.0,4358.0,1173.0,1923.0,474.0,138.0,225.0,7608.0,3077.0,7902.0,1920.0,1169.0,605.0,6733.0,1827.0


In [17]:
# Dropping columns we don't need in both of them

cols_edu = []
for i in range(3,56):
    if i%2!=0:
        cols_edu.append(i)
        
cols_edu_rest = list(range(56,770))

acs_edu = acs_edu.drop(acs_edu.columns[cols_edu_rest], axis = 1)
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu], axis = 1)
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918


In [18]:
# Calculating columns to match the ones we had for our 2016 data
acs_edu['Less than High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years'])
acs_edu['At Least High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)'] + 
    acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])

acs_edu["At Least Bachelor's Degree"] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                                         acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])
acs_edu['Graduate Degree'] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                             acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])


acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686,0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317,0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980,0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474,0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918,0.136021,0.347913,0.081868,0.048427


In [19]:
#Dropping ones we don't need anymore - edu
cols_edu = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu],axis = 1)
acs_edu.head()

,id,Geographic Area Name,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427


In [20]:
#Dropping ones we don't need anymore - race
cols_race = [3,5,7,8,9,11,12,13,14,15,16,17,18,19,20,21]
acs_race = acs_race.drop(acs_race.columns[cols_race], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0


In [21]:
#Converting them to percentages 

acs_race['White (Not Latino) Population'] = acs_race['Estimate!!Total:!!White alone']/acs_race['Estimate!!Total:'] 
acs_race['African American Population'] = acs_race['Estimate!!Total:!!Black or African American alone']/acs_race['Estimate!!Total:']
acs_race['Asian American Population'] = acs_race['Estimate!!Total:!!Asian alone']/acs_race['Estimate!!Total:']

acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0,0.889599,0.007051,0.009063


In [22]:
# Dropping other columns we don't need now
acs_race = acs_race.drop(acs_race.columns[[3,4,5]], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063


In [23]:
# Reading in unemployment data
unemployment = pd.read_csv('data/unemployment2020.csv', header = 4)
unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Sep-19,"26,010","25,391",619,2.4
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Sep-19,"96,754","94,510","2,244",2.3
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Sep-19,"8,656","8,376",280,3.2
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Sep-19,"8,655","8,430",225,2.6


In [24]:
unemployment.Period.unique()

array([nan, 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20',
       'Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20',
       'Sep-20', 'Oct-20 p'], dtype=object)

In [25]:
unemployment.dtypes

LAUS Code                          object
Code                              float64
Code.1                            float64
County Name/State Abbreviation     object
Period                             object
Force                              object
Employed                           object
Unemployed                         object
(%)                                object
dtype: object

In [26]:
# Converting code and code.1 to ints
unemployment = unemployment.fillna(0)
unemployment.Code = unemployment.Code.astype(int)
unemployment['Code.1'] = unemployment['Code.1'].astype(int)

unemployment.dtypes

LAUS Code                         object
Code                               int64
Code.1                             int64
County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
dtype: object

In [27]:
# Getting the data from Oct 2020
unemployment = unemployment[unemployment.Period == 'Oct-20 p']
unemployment = unemployment.reset_index()
unemployment.head()

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3


In [28]:
# Creating a Fips column
unemployment['Fips'] = 0
for i in range(0,len(unemployment)):

    if(unemployment['Code'][i]) < 10:
        if (unemployment['Code.1'][i]) < 10:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
        elif (unemployment['Code.1'][i]) < 100:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
        else:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 10:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 100:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
    else:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
        
unemployment.head()

<ipython-input-28-dbd994a91222>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
<ipython-input-28-dbd994a91222>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
<ipython-input-28-dbd994a91222>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [29]:
# Dropping columns we don't need
unemployment = unemployment.drop(['LAUS Code','Code','Code.1', 'index'], axis = 1)
unemployment.head()

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [30]:
len(unemployment)

3219

In [31]:
# Subsetting by states we needed
counter = 0
for i in range(1,len(unemployment)):
    if(('AZ' in unemployment['County Name/State Abbreviation'][i]) or 
      ('FL' in unemployment['County Name/State Abbreviation'][i]) or 
    ('GA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('NV' in unemployment['County Name/State Abbreviation'][i]) or 
      ('PA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('TX' in unemployment['County Name/State Abbreviation'][i]) or 
      ('IA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('NC' in unemployment['County Name/State Abbreviation'][i]) or 
      ( 'WI' in unemployment['County Name/State Abbreviation'][i]) or 
      ('MI' in unemployment['County Name/State Abbreviation'][i]) or 
      ('OH' in unemployment['County Name/State Abbreviation'][i]) or 
      ('MI' in unemployment['County Name/State Abbreviation'][i])):
        counter+=1
    else:
        unemployment = unemployment.drop([i])
        
unemployment.head()
    

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
96,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001
97,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003
98,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005
99,"Gila County, AZ",Oct-20 p,"20,992","19,379","1,613",7.7,4007


In [32]:
#Reading in poverty data
poverty = pd.read_csv('data/acs_poverty.csv', header = 1)

In [33]:
poverty.head()

,id,Geographic Area Name,Estimate!!Total!!Population for whom poverty status is determined,Margin of Error!!Total!!Population for whom poverty status is determined,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Male,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Male,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Female,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Female,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Estimate!!Total!!Population for whom 

In [34]:
# Getting just the columns that we need and renaming

poverty = poverty[['id','Geographic Area Name','Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED']]
poverty = poverty.rename(columns = {'Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED':'Poverty Percent, All Ages'})
poverty.head()

,id,Geographic Area Name,"Poverty Percent, All Ages"
0,0500000US04001,"Apache County, Arizona",42.8
1,0500000US04003,"Cochise County, Arizona",27.6
2,0500000US04005,"Coconino County, Arizona",38.0
3,0500000US04013,"Maricopa County, Arizona",21.0
4,0500000US04015,"Mohave County, Arizona",29.3


In [35]:
# Reading in 2020 election data
election = pd.read_csv('data/county_statistics.csv')

election.head()

,Unnamed: 0,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,lat,long,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,Abbeville,SC,0.629,0.346,10724.0,6742.0,3712.0,0.661,0.330,12433.0,8215.0,4101.0,34.223334,-82.461707,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4
1,1,Acadia,LA,0.773,0.206,27386.0,21159.0,5638.0,0.795,0.191,28425.0,22596.0,5443.0,30.295065,-92.414197,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9
2,2,Accomack,VA,0.545,0.428,15755.0,8582.0,6737.0,0.542,0.447,16938.0,9172.0,7578.0,37.767072,-75.632346,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4
3,3,Ada,ID,0.479,0.387,195587.0,93748.0,75676.0,0.504,0.465,259389.0,130699.0,120539.0,43.452658,-116.241552,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3
4,4,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,41.330756,-94.471059,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0


In [36]:
#Dropping columns we don't need
cols = [0,5,6,7,13,14]

election = election.drop(election.columns[cols],axis = 1)
election.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,Abbeville,SC,0.629,0.346,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4
1,Acadia,LA,0.773,0.206,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9
2,Accomack,VA,0.545,0.428,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4
3,Ada,ID,0.479,0.387,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3
4,Adair,IA,0.653,0.300,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0


In [37]:
# Creating a column with county and state, formatted like the column in our df2020
election['county_state'] = election['county']+' County, '+election['state']
election.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
0,Abbeville,SC,0.629,0.346,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4,"Abbeville County, SC"
1,Acadia,LA,0.773,0.206,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9,"Acadia County, LA"
2,Accomack,VA,0.545,0.428,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4,"Accomack County, VA"
3,Ada,ID,0.479,0.387,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3,"Ada County, ID"
4,Adair,IA,0.653,0.300,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0,"Adair County, IA"


In [38]:
unemployment.dtypes

County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
Fips                               int64
dtype: object

In [39]:
# converting Fips column in unemployment to strings so we can use it to merge on
unemployment['Fips'] = unemployment['Fips'].apply(str)

In [40]:
# Merge data set for smaller model (~341 counties)
df_small = pd.merge(left = acs_edu, right = acs_race, on = 'id')
df_small = pd.merge(left = df_small, right = poverty, on = 'id')
df_small["id"] = df_small["id"].str.replace("0500000US0", "")
df_small["id"] = df_small["id"].str.replace("0500000US", "")
df_small = pd.merge(left = df_small, right = unemployment, left_on = 'id', right_on = 'Fips', how = 'inner')

df_small.head()


,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015


In [41]:
df_states.dtypes

index                  int64
STATE                  int64
COUNTY                 int64
STNAME                object
CTYNAME               object
AGE18PLUS_TOT          int64
AGE18PLUS_MALE         int64
AGE18PLUS_FEM          int64
AGE1824_TOT            int64
AGE1824_MALE           int64
AGE1824_FEM            int64
AGE2544_TOT            int64
AGE2544_MALE           int64
AGE2544_FEM            int64
AGE4564_TOT            int64
AGE4564_MALE           int64
AGE4564_FEM            int64
AGE65PLUS_TOT          int64
AGE65PLUS_MALE         int64
AGE65PLUS_FEM          int64
1824tot_per          float64
1824M_per            float64
1824F_per            float64
2544tot_per          float64
2544M_per            float64
2544F_per            float64
4564tot_per          float64
4564M_per            float64
4564F_per            float64
65tot_per            float64
65M_per              float64
65F_per              float64
state_county_fips      int64
dtype: object

In [42]:
#Converting to a string so we can merge on it
df_states['state_county_fips'] = df_states['state_county_fips'].apply(str)

In [43]:
#Merging in population broken down by age group data
df_small = pd.merge(left = df_small, right = df_states, left_on = 'id', right_on = 'state_county_fips', how = 'inner')
df_small.head()

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,4001
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,4003
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,4005
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,4013
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015,107,4,15,Arizona,Mohave County,176822,89251,87571,12728,6774,5954,41128,21886,19242,57056,27804,29252,65910,32787,33123,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,4015


In [44]:
df_small

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,4001
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,4003
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,4005
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,4013
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015,107,4,15,Arizona,Mohave County,176822,89251,87571,12728,6774,5954,41128,21886,19242,57056,27804,29252,65910,32787,33123,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,4015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,55127,"Walworth County, Wisconsin",0.086078,0.310001,0.185158,0.099635,"Walworth County, Wisconsin",103868.0,0.941946,0.013633,0.003938,"Walworth County, Wisconsin",30.2,"Walworth County, WI",Oct-20 p,"55,452","52,914","2,538",4.6,55127,779,55,127,Wisconsin,Walworth County,82832,41071,41761,13426,6810,6616,22179,11415,10764,28162,13939,14223,19065,8907,10158,0.162087,0.082215,0.079873,0.267759,0.137809,0.129950,0.339989,0.168280,0.171709,0.230165,0.107531,0.122634,55127
322,55131,"Washington County, Wisconsin",0.052215,0.305443,0.223036,0.085242,"Washington County, Wisconsin",136034.0,0.941029,0.018385,0.019429,"Washington County, Wisconsin",15.3,"Washington County, WI",Oct-20 p,"77,195","73,932","3,263",4.2,55131,803,55,

In [45]:
election.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
0,Abbeville,SC,0.629,0.346,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4,"Abbeville County, SC"
1,Acadia,LA,0.773,0.206,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9,"Acadia County, LA"
2,Accomack,VA,0.545,0.428,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4,"Accomack County, VA"
3,Ada,ID,0.479,0.387,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3,"Ada County, ID"
4,Adair,IA,0.653,0.300,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0,"Adair County, IA"


In [46]:
gdp2019 = pd.read_csv('data/gdp2019.csv', header = 4)
gdp2019.head(20)

,GeoFips,GeoName,LineCode,Description,2019
0,12000,Florida,1.0,All industry total,963255865
1,12000,Florida,2.0,Private industries,863422442
2,12000,Florida,3.0,"Agriculture, forestry, fishing and hunting",8325438
3,12000,Florida,6.0,"Mining, quarrying, and oil and gas extraction",1614201
4,12000,Florida,10.0,Utilities,15812988
5,12000,Florida,11.0,Construction,45066333
6,12000,Florida,12.0,Manufacturing,53976007
7,12000,Florida,13.0,Durable goods manufacturing,33389708
8,12000,Florida,25.0,Nondurable goods manufacturing,20608938
9,12000,Florida,34.0,Wholesale trade,66961976


In [47]:
gdp2019 = gdp2019[gdp2019['Description'] == 'All industry total']
gdp2019.head()

,GeoFips,GeoName,LineCode,Description,2019
0,12000,Florida,1.0,All industry total,963255865
35,13000,Georgia,1.0,All industry total,547422652
70,19000,Iowa,1.0,All industry total,173515422
105,26000,Michigan,1.0,All industry total,471648116
140,27000,Minnesota,1.0,All industry total,341041426


In [48]:
gdp2019 = gdp2019.drop(['LineCode', 'Description'],axis = 1)
gdp2019.head(40)

,GeoFips,GeoName,2019
0,12000,Florida,963255865
35,13000,Georgia,547422652
70,19000,Iowa,173515422
105,26000,Michigan,471648116
140,27000,Minnesota,341041426
175,32000,Nevada,153728626
210,37000,North Carolina,511539864
245,39000,Ohio,615474375
280,42000,Pennsylvania,726165857
315,48000,Texas,1764357169


In [49]:
gdp2019['GeoFips'] = pd.to_numeric(gdp2019['GeoFips'])

In [50]:
gdp2019.dtypes

GeoFips     int64
GeoName    object
2019       object
dtype: object

In [51]:
gdp2019['GeoFips'] = (gdp2019['GeoFips']).apply(str)

In [52]:
gdp2019.dtypes

GeoFips    object
GeoName    object
2019       object
dtype: object

In [53]:
#Merging in GDP
df_small = pd.merge(left = df_small, right = gdp2019, left_on = 'id', right_on = 'GeoFips', how = 'left')

df_small.head()


,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips,GeoFips,GeoName,2019
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,4001,4001,"Apache, AZ",2490196
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,4003,4003,"Cochise, AZ",4589330
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,4005,4005,"Coconino, AZ",6764676
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,4013,4013,"Maricopa, AZ",229821077
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015,107,4,15,Arizona,Mohave County,176822,89251,87571,12728,6774,5954,41128,21886,19242,57056,27804,29252,65910,32787,33123,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,4015,4015,"Mohave, AZ",5200986


In [54]:
sum(df_small['2019'].isna())

0

In [55]:
df_small

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips,GeoFips,GeoName,2019
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,4001,4001,"Apache, AZ",2490196
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,4003,4003,"Cochise, AZ",4589330
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,4005,4005,"Coconino, AZ",6764676
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,4013,4013,"Maricopa, AZ",229821077
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015,107,4,15,Arizona,Mohave County,176822,89251,87571,12728,6774,5954,41128,21886,19242,57056,27804,29252,65910,32787,33123,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,4015,4015,"Mohave, AZ",5200986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,55127,"Walworth County, Wisconsin",0.086078,0.310001,0.185158,0.099635,"Walworth County, Wisconsin",103868.0,0.941946,0.013633,0.003938,"Walworth County, Wisconsin",30.2,"Walworth County, WI",Oct-20 p,"55,452","52,914","2,538",4.6,55127,779,55,127,Wisconsin,Walworth County,82832,41071,41761,13426,6810,6616,22179,11415,10764,28162,13939,14223,19065,8907,10158,0.162087,0.082215,0.079873,0.267759,0.137809,0.129950,0.339989,0.168280,0.171709,0.230165,0.107531,0.122634,55127,55127,"Walworth, WI",3974747
322,55131,"Washington County, Wisconsin",0.052215,0.30544

In [56]:
#Filtering for just the states we want
counter = 0
for i in range(0,len(election)):
    if(('AZ' in election['state'][i]) or 
      ('FL' in election['state'][i]) or 
    ('GA' in election['state'][i]) or 
      ('NV' in election['state'][i]) or 
      ('PA' in election['state'][i]) or 
      ('TX' in election['state'][i]) or 
      ('IA' in election['state'][i]) or 
      ('NC' in election['state'][i]) or 
      ( 'WI' in election['state'][i]) or 
      ('MI' in election['state'][i]) or 
      ('OH' in election['state'][i]) or 
      ('MN' in election['state'][i])):
        counter+=1
    else:
        election = election.drop([i])
        
election.head()
    

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
4,Adair,IA,0.653,0.300,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0,"Adair County, IA"
9,Adams,IA,0.669,0.271,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,3785.0,1870.0,1915.0,1.2,96.8,0.4,0.1,0.6,0.0,2992.0,49745.0,3996.0,27022.0,1948.0,11.2,8.6,37.4,14.3,17.0,13.1,18.3,80.7,9.6,0.2,3.3,0.8,5.3,19.6,1796.0,72.0,12.1,15.5,0.3,4.2,"Adams County, IA"
16,Adams,OH,0.763,0.207,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,27926.0,13960.0,13966.0,1.0,96.6,0.4,0.6,0.2,0.0,21041.0,36320.0,2651.0,20248.0,1164.0,23.8,32.0,28.3,18.9,16.7,13.6,22.5,80.8,11.5,0.5,1.3,1.2,4.8,36.5,10124.0,79.1,13.0,7.7,0.2,7.5,"Adams County, OH"
17,Adams,PA,0.663,0.299,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,101589.0,50055.0,51534.0,6.8,89.5,1.4,0.0,0.7,0.0,78227.0,62661.0,1660.0,29685.0,663.0,8.8,13.9,30.1,16.2,22.1,11.0,20.6,82.3,9.1,0.3,3.7,0.9,3.8,27.6,50561.0,83.4,10.7,5.6,0.2,4.9,"Adams County, PA"
19,Adams,WI,0.592,0.374,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,20111.0,10770.0,9341.0,3.8,90.7,2.7,0.6,0.5,0.0,16837.0,44003.0,1500.0,24874.0,1297.0,13.0,23.3,23.6,22.7,21.8,12.5,19.4,78.5,12.2,0.1,1.5,1.7,6.0,28.3,7530.0,80.7,11.9,7.3,0.2,8.1,"Adams County, WI"


In [57]:
election

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
4,Adair,IA,0.653,0.300,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0,"Adair County, IA"
9,Adams,IA,0.669,0.271,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,3785.0,1870.0,1915.0,1.2,96.8,0.4,0.1,0.6,0.0,2992.0,49745.0,3996.0,27022.0,1948.0,11.2,8.6,37.4,14.3,17.0,13.1,18.3,80.7,9.6,0.2,3.3,0.8,5.3,19.6,1796.0,72.0,12.1,15.5,0.3,4.2,"Adams County, IA"
16,Adams,OH,0.763,0.207,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,27926.0,13960.0,13966.0,1.0,96.6,0.4,0.6,0.2,0.0,21041.0,36320.0,2651.0,20248.0,1164.0,23.8,32.0,28.3,18.9,16.7,13.6,22.5,80.8,11.5,0.5,1.3,1.2,4.8,36.5,10124.0,79.1,13.0,7.7,0.2,7.5,"Adams County, OH"
17,Adams,PA,0.663,0.299,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,101589.0,50055.0,51534.0,6.8,89.5,1.4,0.0,0.7,0.0,78227.0,62661.0,1660.0,29685.0,663.0,8.8,13.9,30.1,16.2,22.1,11.0,20.6,82.3,9.1,0.3,3.7,0.9,3.8,27.6,50561.0,83.4,10.7,5.6,0.2,4.9,"Adams County, PA"
19,Adams,WI,0.592,0.374,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,20111.0,10770.0,9341.0,3.8,90.7,2.7,0.6,0.5,0.0,16837.0,44003.0,1500.0,24874.0,1297.0,13.0,23.3,23.6,22.7,21.8,12.5,19.4,78.5,12.2,0.1,1.5,1.7,6.0,28.3,7530.0,80.7,11.9,7.3,0.2,8.1,"Adams County, WI"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,Unassigned,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Unassigned County, PA"
4777,Out of TX,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Out of TX County, TX"
4778,Unassigned,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Unassigned County, TX"
4833,Out of WI,WI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Out of WI County, WI"


In [58]:
# Merging in 2020 election data
df_small = pd.merge(left = df_small, right = election, left_on = 'County Name/State Abbreviation', right_on = 'county_state', how = 'inner')
df_small

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed_x,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,...,state_county_fips,GeoFips,GeoName,2019,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed_y,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,...,4001,4001,"Apache, AZ",2490196,Apache,AZ,0.285,0.654,0.327,0.661,34397.0,11240.0,22730.0,4003.0,171.0,71602.0,35355.0,36247.0,6.1,18.5,0.5,72.8,0.3,0.1,50359.0,32360.0,1213.0,13865.0,370.0,35.9,45.3,29.7,25.4,20.6,11.5,12.8,78.0,8.9,1.2,4.6,1.2,6.2,26.7,18197.0,52.7,42.7,4.4,0.1,14.1,"Apache County, AZ"
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,...,4003,4003,"Cochise, AZ",4589330,Cochise,AZ,0.580,0.354,0.582,0.399,57672.0,33580.0,22997.0,2205.0,75.0,126516.0,64094.0,62422.0,35.0,55.7,3.6,0.9,1.8,0.4,92064.0,47847.0,1539.0,24896.0,602.0,18.1,25.9,34.4,24.3,24.1,10.1,7.2,79.1,10.3,0.7,2.8,2.7,4.4,20.5,43404.0,63.6,29.9,6.1,0.4,7.6,"Cochise County, AZ"
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,...,4005,4005,"Coconino, AZ",6764676,Coconino,AZ,0.369,0.563,0.370,0.610,73174.0,27043.0,44609.0,5405.0,149.0,138639.0,68264.0,70375.0,13.9,54.3,1.2,25.8,1.8,0.1,105097.0,53523.0,2500.0,25722.0,1041.0,21.0,23.2,36.5,22.7,22.5,8.4,9.9,70.0,11.4,1.5,8.6,3.8,4.8,18.4,65283.0,71.5,23.5,4.7,0.2,8.3,"Coconino County, AZ"
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,...,4013,4013,"Maricopa, 

In [59]:
df_small.state.unique()

array(['AZ', 'FL', 'GA', 'IA', 'MI', 'NV', 'NC', 'OH', 'PA', 'TX', 'WI'],
      dtype=object)

In [60]:
# Converting GDP to log GDP as we did in the 2016 model
df_small['2019'] = pd.to_numeric(df_small['2019'])
df_small['log_gdp'] = np.log10(df_small['2019'])

In [61]:
df_small.head()

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed_x,Unemployed,(%),Fips,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,...,GeoFips,GeoName,2019,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed_y,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state,log_gdp
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001,11,4,1,Arizona,Apache County,52462,25766,26696,6513,3222,3291,17402,9129,8273,17213,8286,8927,11334,5129,6205,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,...,4001,"Apache, AZ",2490196,Apache,AZ,0.285,0.654,0.327,0.661,34397.0,11240.0,22730.0,4003.0,171.0,71602.0,35355.0,36247.0,6.1,18.5,0.5,72.8,0.3,0.1,50359.0,32360.0,1213.0,13865.0,370.0,35.9,45.3,29.7,25.4,20.6,11.5,12.8,78.0,8.9,1.2,4.6,1.2,6.2,26.7,18197.0,52.7,42.7,4.4,0.1,14.1,"Apache County, AZ",6.396234
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003,23,4,3,Arizona,Cochise County,98949,50166,48783,10204,5722,4482,29774,16041,13733,29853,14511,15342,29118,13892,15226,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,...,4003,"Cochise, AZ",4589330,Cochise,AZ,0.580,0.354,0.582,0.399,57672.0,33580.0,22997.0,2205.0,75.0,126516.0,64094.0,62422.0,35.0,55.7,3.6,0.9,1.8,0.4,92064.0,47847.0,1539.0,24896.0,602.0,18.1,25.9,34.4,24.3,24.1,10.1,7.2,79.1,10.3,0.7,2.8,2.7,4.4,20.5,43404.0,63.6,29.9,6.1,0.4,7.6,"Cochise County, AZ",6.661749
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005,35,4,5,Arizona,Coconino County,114726,55923,58803,29622,13481,16141,36060,18675,17385,30382,14876,15506,18662,8891,9771,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,...,4005,"Coconino, AZ",6764676,Coconino,AZ,0.369,0.563,0.370,0.610,73174.0,27043.0,44609.0,5405.0,149.0,138639.0,68264.0,70375.0,13.9,54.3,1.2,25.8,1.8,0.1,105097.0,53523.0,2500.0,25722.0,1041.0,21.0,23.2,36.5,22.7,22.5,8.4,9.9,70.0,11.4,1.5,8.6,3.8,4.8,18.4,65283.0,71.5,23.5,4.7,0.2,8.3,"Coconino County, AZ",6.830247
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013,95,4,13,Arizona,Maricopa County,3432976,1680553,1752423,415938,213070,202868,1242721,628109,614612,1077986,525456,552530,696331,313918,382413,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,...,4013,"Maricopa, AZ"

In [62]:
# Final subset of features to export for our model construction
df_small_final = df_small[['Less than High School','At Least High School', 'White (Not Latino) Population',
                    'African American Population','Asian American Population','(%)','Poverty Percent, All Ages',
                    'Graduate Degree', "At Least Bachelor's Degree", '1824tot_per','1824M_per',
             '1824F_per','2544tot_per','2544M_per','2544F_per','4564tot_per','4564M_per','4564F_per','65tot_per',
             '65M_per','65F_per','percentage16_Donald_Trump', 'percentage16_Hillary_Clinton','log_gdp',
                     'Geographic Area Name_x',
                    'id','percentage20_Donald_Trump','percentage20_Joe_Biden']]
df_small_final.head()

,Less than High School,At Least High School,White (Not Latino) Population,African American Population,Asian American Population,(%),"Poverty Percent, All Ages",Graduate Degree,At Least Bachelor's Degree,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,percentage16_Donald_Trump,percentage16_Hillary_Clinton,log_gdp,Geographic Area Name_x,id,percentage20_Donald_Trump,percentage20_Joe_Biden
0,0.179969,0.314415,0.210177,0.010030,0.002226,12.7,42.8,0.053024,0.056851,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,0.285,0.654,6.396234,"Apache County, Arizona",4001,0.327,0.661
1,0.123738,0.271272,0.859079,0.048943,0.022196,6.6,27.6,0.089437,0.135565,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,0.580,0.354,6.661749,"Cochise County, Arizona",4003,0.582,0.399
2,0.058941,0.224030,0.638776,0.009507,0.019028,9.7,38.0,0.145664,0.178253,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,0.369,0.563,6.830247,"Coconino County, Arizona",4005,0.370,0.610
3,0.121441,0.239276,0.790820,0.059332,0.042873,7.5,21.0,0.122930,0.201831,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,0.491,0.457,8.361390,"Maricopa County, Arizona",4013,0.482,0.503
4,0.136021,0.347913,0.889599,0.007051,0.009063,9.1,29.3,0.048427,0.081868,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,0.737,0.222,6.716086,"Mohave County, Arizona",4015,0.750,0.237


In [63]:
#Renaming columns to match our 2016 models
df_small_final = df_small_final.rename(columns = {'percentage16_Donald_Trump':'Republicans Previous Year', 'percentage16_Hillary_Clinton':'Democrats Previous Year',
                         'Less than High School':'Less Than High School Diploma','At Least High School':'At Least High School Diploma',
                         '(%)':'Average Unemployment %', "At Least Bachelor's Degree":"At Least Bachelors's Degree", 
                                                 'percentage20_Donald_Trump':'Republicans Target Year',
                                                 'percentage20_Joe_Biden':'Democrats Target Year'})
df_small_final.head()

,Less Than High School Diploma,At Least High School Diploma,White (Not Latino) Population,African American Population,Asian American Population,Average Unemployment %,"Poverty Percent, All Ages",Graduate Degree,At Least Bachelors's Degree,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,Republicans Previous Year,Democrats Previous Year,log_gdp,Geographic Area Name_x,id,Republicans Target Year,Democrats Target Year
0,0.179969,0.314415,0.210177,0.010030,0.002226,12.7,42.8,0.053024,0.056851,0.124147,0.061416,0.062731,0.331707,0.174012,0.157695,0.328104,0.157943,0.170161,0.216042,0.097766,0.118276,0.285,0.654,6.396234,"Apache County, Arizona",4001,0.327,0.661
1,0.123738,0.271272,0.859079,0.048943,0.022196,6.6,27.6,0.089437,0.135565,0.103124,0.057828,0.045296,0.300902,0.162114,0.138789,0.301701,0.146651,0.155050,0.294273,0.140396,0.153877,0.580,0.354,6.661749,"Cochise County, Arizona",4003,0.582,0.399
2,0.058941,0.224030,0.638776,0.009507,0.019028,9.7,38.0,0.145664,0.178253,0.258198,0.117506,0.140692,0.314314,0.162779,0.151535,0.264822,0.129665,0.135157,0.162666,0.077498,0.085168,0.369,0.563,6.830247,"Coconino County, Arizona",4005,0.370,0.610
3,0.121441,0.239276,0.790820,0.059332,0.042873,7.5,21.0,0.122930,0.201831,0.121160,0.062066,0.059094,0.361995,0.182963,0.179032,0.314009,0.153061,0.160948,0.202836,0.091442,0.111394,0.491,0.457,8.361390,"Maricopa County, Arizona",4013,0.482,0.503
4,0.136021,0.347913,0.889599,0.007051,0.009063,9.1,29.3,0.048427,0.081868,0.071982,0.038310,0.033672,0.232595,0.123774,0.108821,0.322675,0.157243,0.165432,0.372748,0.185424,0.187324,0.737,0.222,6.716086,"Mohave County, Arizona",4015,0.750,0.237


In [64]:

df_small_final.to_csv(r'/Users/emilyweed/Cornell2020-2021/ORIE4741/Project/ORIE-4741-Project/data/exported_data2020small.csv', index=False, header = True)


In [65]:
# create data set to use for the model dropping the education, 
# using the race data from the kaggle set



In [66]:
df_orig = pd.read_csv('data/county_statistics.csv')
df_orig.head()

,Unnamed: 0,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,lat,long,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,Abbeville,SC,0.629,0.346,10724.0,6742.0,3712.0,0.661,0.330,12433.0,8215.0,4101.0,34.223334,-82.461707,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4
1,1,Acadia,LA,0.773,0.206,27386.0,21159.0,5638.0,0.795,0.191,28425.0,22596.0,5443.0,30.295065,-92.414197,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9
2,2,Accomack,VA,0.545,0.428,15755.0,8582.0,6737.0,0.542,0.447,16938.0,9172.0,7578.0,37.767072,-75.632346,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4
3,3,Ada,ID,0.479,0.387,195587.0,93748.0,75676.0,0.504,0.465,259389.0,130699.0,120539.0,43.452658,-116.241552,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3
4,4,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,41.330756,-94.471059,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0


In [67]:
#Dropping columns we dont need
cols = [0,13,14,17,18,19,20,23,25,26,27,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]
df_orig = df_orig.drop(df_orig.columns[cols],axis=1)

In [68]:
df_orig.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment
0,Abbeville,SC,0.629,0.346,10724.0,6742.0,3712.0,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,68.9,27.6,0.3,22.7,9.4
1,Acadia,LA,0.773,0.206,27386.0,21159.0,5638.0,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,77.5,17.6,0.1,21.5,8.9
2,Accomack,VA,0.545,0.428,15755.0,8582.0,6737.0,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,60.3,28.3,0.7,19.8,5.4
3,Ada,ID,0.479,0.387,195587.0,93748.0,75676.0,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,85.2,1.2,2.6,11.8,4.3
4,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0


In [69]:
unemployment = pd.read_csv('data/unemployment2020.csv', header = 4)
unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Sep-19,"26,010","25,391",619,2.4
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Sep-19,"96,754","94,510","2,244",2.3
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Sep-19,"8,656","8,376",280,3.2
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Sep-19,"8,655","8,430",225,2.6


In [70]:
#converting code and code.1 to ints so we can make a FIPS column
unemployment = unemployment.fillna(0)
unemployment.Code = unemployment.Code.astype(int)
unemployment['Code.1'] = unemployment['Code.1'].astype(int)

In [71]:
#Dropping columns we don't need
cols_todrop = [0,5,6,7,8]
unemployment = unemployment.drop(unemployment.columns[cols_todrop], axis=1)

In [72]:
#Subsetting by date we needed
unemployment = unemployment[unemployment.Period == 'Oct-20 p']
unemployment = unemployment.reset_index()
unemployment.head()

,index,Code,Code.1,County Name/State Abbreviation,Period
0,41848,1,1,"Autauga County, AL",Oct-20 p
1,41849,1,3,"Baldwin County, AL",Oct-20 p
2,41850,1,5,"Barbour County, AL",Oct-20 p
3,41851,1,7,"Bibb County, AL",Oct-20 p
4,41852,1,9,"Blount County, AL",Oct-20 p


In [73]:
#Adding in Fips column
unemployment['Fips'] = 0
for i in range(0,len(unemployment)):

    if(unemployment['Code'][i]) < 10:
        if (unemployment['Code.1'][i]) < 10:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
        elif (unemployment['Code.1'][i]) < 100:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
        else:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 10:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 100:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
    else:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
        
unemployment.head()

<ipython-input-73-910fba72e91b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
<ipython-input-73-910fba72e91b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
<ipython-input-73-910fba72e91b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,Code,Code.1,County Name/State Abbreviation,Period,Fips
0,41848,1,1,"Autauga County, AL",Oct-20 p,1001
1,41849,1,3,"Baldwin County, AL",Oct-20 p,1003
2,41850,1,5,"Barbour County, AL",Oct-20 p,1005
3,41851,1,7,"Bibb County, AL",Oct-20 p,1007
4,41852,1,9,"Blount County, AL",Oct-20 p,1009


In [74]:
#Dropping more columns
unemployment = unemployment.drop(unemployment.columns[[0,1,2,4]],axis=1)
unemployment.head()

,County Name/State Abbreviation,Fips
0,"Autauga County, AL",1001
1,"Baldwin County, AL",1003
2,"Barbour County, AL",1005
3,"Bibb County, AL",1007
4,"Blount County, AL",1009


In [75]:
#Creating column to join on
df_orig['county_state'] = df_orig['county']+' County, '+ df_orig['state']

In [76]:
# Merging unemployment and our original
# this gives us the fips column so we can use it later to join on
df_full = pd.merge(left=df_orig, right=unemployment, left_on="county_state", right_on="County Name/State Abbreviation", how="inner")

df_full.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment,county_state,County Name/State Abbreviation,Fips
0,Abbeville,SC,0.629,0.346,10724.0,6742.0,3712.0,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,68.9,27.6,0.3,22.7,9.4,"Abbeville County, SC","Abbeville County, SC",45001
1,Accomack,VA,0.545,0.428,15755.0,8582.0,6737.0,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,60.3,28.3,0.7,19.8,5.4,"Accomack County, VA","Accomack County, VA",51001
2,Ada,ID,0.479,0.387,195587.0,93748.0,75676.0,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,85.2,1.2,2.6,11.8,4.3,"Ada County, ID","Ada County, ID",16001
3,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0,"Adair County, IA","Adair County, IA",19001
4,Adair,KY,0.806,0.161,8231.0,6637.0,1323.0,0.830,0.159,8766.0,7275.0,1391.0,517.0,22.0,93.4,3.6,0.1,21.5,6.2,"Adair County, KY","Adair County, KY",21001


In [77]:
#Reading in population by age data
state13 = pd.read_csv('data/cc-est2019-agesex-13.csv')
state19 = pd.read_csv('data/cc-est2019-agesex-19.csv')
state12 = pd.read_csv('data/cc-est2019-agesex-12.csv')
state27 = pd.read_csv('data/cc-est2019-agesex-27.csv')
state32 = pd.read_csv('data/cc-est2019-agesex-32.csv')
state37 = pd.read_csv('data/cc-est2019-agesex-37.csv')
state39 = pd.read_csv('data/cc-est2019-agesex-39.csv')
state42 = pd.read_csv('data/cc-est2019-agesex-42.csv')
state48 = pd.read_csv('data/cc-est2019-agesex-48.csv')
state55 = pd.read_csv('data/cc-est2019-agesex-55.csv')
state26 = pd.read_csv('data/cc-est2019-agesex-26.csv')
state4 = pd.read_csv('data/cc-est2019-agesex-04.csv')
df_states = state13.append(state19).append(state12).append(state27).append(state32).append(state37).append(state39).append(state42).append(state48).append(state55).append(state26).append(state4)

df_states = df_states[df_states['YEAR'] == 12] #12 represents 2019 (as close to 2020 as we could find)

In [78]:
#Subsetting by columns we need
df_states = df_states[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','AGE18PLUS_TOT','AGE18PLUS_MALE','AGE18PLUS_FEM', 'AGE1824_TOT', 'AGE1824_MALE','AGE1824_FEM', 
                       'AGE2544_TOT','AGE2544_MALE','AGE2544_FEM', 'AGE4564_TOT','AGE4564_MALE', 'AGE4564_FEM',
                      'AGE65PLUS_TOT', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM']]

In [79]:
# Dividing by total value to get a percentage 
df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
df_states['1824F_per'] = df_states['AGE1824_FEM']/df_states['AGE18PLUS_TOT']
df_states['2544tot_per'] = df_states['AGE2544_TOT']/df_states['AGE18PLUS_TOT']
df_states['2544M_per'] = df_states['AGE2544_MALE']/df_states['AGE18PLUS_TOT']
df_states['2544F_per'] = df_states['AGE2544_FEM']/df_states['AGE18PLUS_TOT']
df_states['4564tot_per'] = df_states['AGE4564_TOT']/df_states['AGE18PLUS_TOT']
df_states['4564M_per'] = df_states['AGE4564_MALE']/df_states['AGE18PLUS_TOT']
df_states['4564F_per'] = df_states['AGE4564_FEM']/df_states['AGE18PLUS_TOT']
df_states['65tot_per'] = df_states['AGE65PLUS_TOT']/df_states['AGE18PLUS_TOT']
df_states['65M_per'] = df_states['AGE65PLUS_MALE']/df_states['AGE18PLUS_TOT']
df_states['65F_per'] = df_states['AGE65PLUS_FEM']/df_states['AGE18PLUS_TOT']

In [80]:
#Creating a fips column so we can merge
df_states['state_county_fips'] = 0
df_states = df_states.reset_index()
for i in range(0,len(df_states)):
    if(df_states['STATE'][i]) < 10:
        if (df_states['COUNTY'][i]) < 10:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
        elif (df_states['COUNTY'][i]) < 100:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '0' + str(df_states['COUNTY'][i])
        else:
            df_states['state_county_fips'][i] = '0'+str(df_states['STATE'][i]) + '' + str(df_states['COUNTY'][i])
    elif (df_states['COUNTY'][i]) < 10:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
    elif (df_states['COUNTY'][i]) < 100:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '0' + str(df_states['COUNTY'][i])
    else:
        df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '' + str(df_states['COUNTY'][i])
df_states.head()

<ipython-input-80-b912df862422>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '00' + str(df_states['COUNTY'][i])
<ipython-input-80-b912df862422>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['state_county_fips'][i] = str(df_states['STATE'][i]) + '0' + str(df_states['COUNTY'][i])
<ipython-input-80-b912df862422>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,index,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235,13001
1,23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245,13003
2,35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596,13005
3,47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701,13007
4,59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109,13009


In [81]:
#Converting it to a string
df_states['state_county_fips'] = df_states['state_county_fips'].apply(str)

In [82]:
drop_state_cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
df_states = df_states.drop(df_states.columns[drop_state_cols], axis=1)
df_states.head()

,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235,13001
1,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245,13003
2,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596,13005
3,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701,13007
4,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109,13009


In [83]:
#converting fips to a string so we can join on it
df_full['Fips'] = df_full['Fips'].apply(str)

In [84]:
# joining
df_full = pd.merge(left=df_full, right=df_states, left_on="Fips", right_on="state_county_fips", how="inner")
df_full.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment,county_state,County Name/State Abbreviation,Fips,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips
0,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0,"Adair County, IA","Adair County, IA",19001,0.081173,0.042732,0.038441,0.286251,0.146612,0.139639,0.337922,0.172895,0.165028,0.294654,0.130341,0.164313,19001
1,Adams,IA,0.669,0.271,2082.0,1393.0,565.0,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,96.8,0.4,0.6,11.2,4.2,"Adams County, IA","Adams County, IA",19003,0.077598,0.042135,0.035463,0.266503,0.135534,0.130969,0.356742,0.179424,0.177317,0.299157,0.141854,0.157303,19003
2,Adams,OH,0.763,0.207,11063.0,8445.0,2293.0,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,96.6,0.4,0.2,23.8,7.5,"Adams County, OH","Adams County, OH",39001,0.097295,0.048079,0.049216,0.295059,0.143859,0.151201,0.365497,0.182227,0.183269,0.242149,0.114016,0.128132,39001
3,Adams,PA,0.663,0.299,47138.0,31249.0,14077.0,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,89.5,1.4,0.7,8.8,4.9,"Adams County, PA","Adams County, PA",42001,0.115262,0.057970,0.057292,0.268496,0.136554,0.131942,0.353205,0.173486,0.179720,0.263037,0.120104,0.142933,42001
4,Adams,WI,0.592,0.374,10107.0,5983.0,3780.0,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,90.7,2.7,0.5,13.0,8.1,"Adams County, WI","Adams County, WI",55001,0.063148,0.035920,0.027228,0.220815,0.133145,0.087670,0.365013,0.190134,0.174879,0.351025,0.176836,0.174188,55001


In [85]:
gdp2019.head()

,GeoFips,GeoName,2019
0,12000,Florida,963255865
35,13000,Georgia,547422652
70,19000,Iowa,173515422
105,26000,Michigan,471648116
140,27000,Minnesota,341041426


In [86]:
#Merging in GDP
df_full = pd.merge(left = df_full, right = gdp2019, left_on = 'Fips', right_on = 'GeoFips', how = 'left')
df_full.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment,county_state,County Name/State Abbreviation,Fips,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips,GeoFips,GeoName,2019
0,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0,"Adair County, IA","Adair County, IA",19001,0.081173,0.042732,0.038441,0.286251,0.146612,0.139639,0.337922,0.172895,0.165028,0.294654,0.130341,0.164313,19001,19001,"Adair, IA",409441
1,Adams,IA,0.669,0.271,2082.0,1393.0,565.0,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,96.8,0.4,0.6,11.2,4.2,"Adams County, IA","Adams County, IA",19003,0.077598,0.042135,0.035463,0.266503,0.135534,0.130969,0.356742,0.179424,0.177317,0.299157,0.141854,0.157303,19003,19003,"Adams, IA",248585
2,Adams,OH,0.763,0.207,11063.0,8445.0,2293.0,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,96.6,0.4,0.2,23.8,7.5,"Adams County, OH","Adams County, OH",39001,0.097295,0.048079,0.049216,0.295059,0.143859,0.151201,0.365497,0.182227,0.183269,0.242149,0.114016,0.128132,39001,39001,"Adams, OH",672809
3,Adams,PA,0.663,0.299,47138.0,31249.0,14077.0,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,89.5,1.4,0.7,8.8,4.9,"Adams County, PA","Adams County, PA",42001,0.115262,0.057970,0.057292,0.268496,0.136554,0.131942,0.353205,0.173486,0.179720,0.263037,0.120104,0.142933,42001,42001,"Adams, PA",3540312
4,Adams,WI,0.592,0.374,10107.0,5983.0,3780.0,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,90.7,2.7,0.5,13.0,8.1,"Adams County, WI","Adams County, WI",55001,0.063148,0.035920,0.027228,0.220815,0.133145,0.087670,0.365013,0.190134,0.174879,0.351025,0.176836,0.174188,55001,55001,"Adams, WI",469877


In [87]:
df_full['2019'] = pd.to_numeric(df_full['2019'])
df_full['log_gdp'] = np.log10(df_full['2019'])
df_full.head()

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment,county_state,County Name/State Abbreviation,Fips,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips,GeoFips,GeoName,2019,log_gdp
0,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0,"Adair County, IA","Adair County, IA",19001,0.081173,0.042732,0.038441,0.286251,0.146612,0.139639,0.337922,0.172895,0.165028,0.294654,0.130341,0.164313,19001,19001,"Adair, IA",409441,5.612191
1,Adams,IA,0.669,0.271,2082.0,1393.0,565.0,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,96.8,0.4,0.6,11.2,4.2,"Adams County, IA","Adams County, IA",19003,0.077598,0.042135,0.035463,0.266503,0.135534,0.130969,0.356742,0.179424,0.177317,0.299157,0.141854,0.157303,19003,19003,"Adams, IA",248585,5.395475
2,Adams,OH,0.763,0.207,11063.0,8445.0,2293.0,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,96.6,0.4,0.2,23.8,7.5,"Adams County, OH","Adams County, OH",39001,0.097295,0.048079,0.049216,0.295059,0.143859,0.151201,0.365497,0.182227,0.183269,0.242149,0.114016,0.128132,39001,39001,"Adams, OH",672809,5.827892
3,Adams,PA,0.663,0.299,47138.0,31249.0,14077.0,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,89.5,1.4,0.7,8.8,4.9,"Adams County, PA","Adams County, PA",42001,0.115262,0.057970,0.057292,0.268496,0.136554,0.131942,0.353205,0.173486,0.179720,0.263037,0.120104,0.142933,42001,42001,"Adams, PA",3540312,6.549042
4,Adams,WI,0.592,0.374,10107.0,5983.0,3780.0,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,90.7,2.7,0.5,13.0,8.1,"Adams County, WI","Adams County, WI",55001,0.063148,0.035920,0.027228,0.220815,0.133145,0.087670,0.365013,0.190134,0.174879,0.351025,0.176836,0.174188,55001,55001,"Adams, WI",469877,5.671984


In [88]:
df_full

,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,White,Black,Asian,Poverty,Unemployment,county_state,County Name/State Abbreviation,Fips,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,state_county_fips,GeoFips,GeoName,2019,log_gdp
0,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,96.6,0.3,0.4,9.5,3.0,"Adair County, IA","Adair County, IA",19001,0.081173,0.042732,0.038441,0.286251,0.146612,0.139639,0.337922,0.172895,0.165028,0.294654,0.130341,0.164313,19001,19001,"Adair, IA",409441,5.612191
1,Adams,IA,0.669,0.271,2082.0,1393.0,565.0,0.708,0.273,2158.0,1528.0,590.0,87.0,1.0,96.8,0.4,0.6,11.2,4.2,"Adams County, IA","Adams County, IA",19003,0.077598,0.042135,0.035463,0.266503,0.135534,0.130969,0.356742,0.179424,0.177317,0.299157,0.141854,0.157303,19003,19003,"Adams, IA",248585,5.395475
2,Adams,OH,0.763,0.207,11063.0,8445.0,2293.0,0.811,0.179,11905.0,9660.0,2130.0,270.0,6.0,96.6,0.4,0.2,23.8,7.5,"Adams County, OH","Adams County, OH",39001,0.097295,0.048079,0.049216,0.295059,0.143859,0.151201,0.365497,0.182227,0.183269,0.242149,0.114016,0.128132,39001,39001,"Adams, OH",672809,5.827892
3,Adams,PA,0.663,0.299,47138.0,31249.0,14077.0,0.663,0.322,56627.0,37563.0,18250.0,1184.0,31.0,89.5,1.4,0.7,8.8,4.9,"Adams County, PA","Adams County, PA",42001,0.115262,0.057970,0.057292,0.268496,0.136554,0.131942,0.353205,0.173486,0.179720,0.263037,0.120104,0.142933,42001,42001,"Adams, PA",3540312,6.549042
4,Adams,WI,0.592,0.374,10107.0,5983.0,3780.0,0.623,0.366,11818.0,7362.0,4329.0,587.0,5.0,90.7,2.7,0.5,13.0,8.1,"Adams County, WI","Adams County, WI",55001,0.063148,0.035920,0.027228,0.220815,0.133145,0.087670,0.365013,0.190134,0.174879,0.351025,0.176836,0.174188,55001,55001,"Adams, WI",469877,5.671984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,York,PA,0.625,0.332,203153.0,126933.0,67428.0,0.615,0.367,233060.0,143415.0,85491.0,6742.0,207.0,84.3,5.2,1.4,10.3,5.4,"York County, PA","York County, PA",42133,0.102107,0.052502,0.049605,0.315306,0.158294,0.157012,0.352454,0.173864,0.178590,0.230133,0.103971,0.126163,42133,42133,"York, PA",19163414,7.282473
1102,Young,TX,0.860,0.114,7658.0,6587.0,873.0,0.863,0.126,8242.0,7112.0,1036.0,620.0,11.0,78.8,1.5,0.4,16.5,4.5,"Young County, TX","Young County, TX",48503,0.094843,0.051495,0.043349,0.296458,0.147283,0.149174,0.334206,0.167794,0.166412,0.274493,0.124518,0.149975,48503,48503,"Young, TX",893312,5.951003
1103,Yuma,AZ,0.505,0.450,40759.0,20586.0,18336.0,0.523,0.461,68427.0,35766.0,31562.0,13994.0,358.0,31.8,1.8,1.1,19.7,10.9,"Yuma County, AZ","Yuma County, AZ",4027,0.146681,0.086101,0.060580,0.333340,0.180897,0.152442,0.262193,0.127170,0.135022,0.257786,0.122957,0.134829,4027,4027,"Yuma, AZ",7926771,6.899096
1104,Zapata,TX,0.328,0.656,3134.0,1028.0,2056.0,0.525,0.471,3867.0,2032.0,1820.0,358.0,9.0,5.2,0.0,0.7,34.1,12.4,"Zapata County, TX","Zapata County, TX",48505,0.155937,0.080813,0.075124,0.360131,0.175956,0.184174,0.287325,0.133706,0.153619,0.196607,0.090823,0.105784,48505,48505,"Zapata, TX",861213,5.935111


In [89]:
#Renaming columns to match the 2016 data names
df_full = df_full.rename(columns = {'percentage16_Donald_Trump':'Republicans Previous Year', 'percentage16_Hillary_Clinton':'Democrats Previous Year',
                         'Unemployment':'Average Unemployment %', 'White' : 'White (Not Latino) Population', 'Black' : 'African American Population',  
                                                 'Asian':'Asian American Population','percentage20_Donald_Trump':'Republicans Target Year', 'Poverty':'Poverty Percent, All Ages',
                                                 'percentage20_Joe_Biden':'Democrats Target Year'})

In [90]:
#Dropping last columns to get final feature subset
cols_to_drop = [0,1,4,5,6,9,10,11,12,13,19,20,21,34,35,36,37]
df_full2 = df_full.drop(df_full.columns[cols_to_drop],axis=1)
df_full2.head()


,Republicans Previous Year,Democrats Previous Year,Republicans Target Year,Democrats Target Year,White (Not Latino) Population,African American Population,Asian American Population,"Poverty Percent, All Ages",Average Unemployment %,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per,log_gdp
0,0.653,0.300,0.697,0.286,96.6,0.3,0.4,9.5,3.0,0.081173,0.042732,0.038441,0.286251,0.146612,0.139639,0.337922,0.172895,0.165028,0.294654,0.130341,0.164313,5.612191
1,0.669,0.271,0.708,0.273,96.8,0.4,0.6,11.2,4.2,0.077598,0.042135,0.035463,0.266503,0.135534,0.130969,0.356742,0.179424,0.177317,0.299157,0.141854,0.157303,5.395475
2,0.763,0.207,0.811,0.179,96.6,0.4,0.2,23.8,7.5,0.097295,0.048079,0.049216,0.295059,0.143859,0.151201,0.365497,0.182227,0.183269,0.242149,0.114016,0.128132,5.827892
3,0.663,0.299,0.663,0.322,89.5,1.4,0.7,8.8,4.9,0.115262,0.057970,0.057292,0.268496,0.136554,0.131942,0.353205,0.173486,0.179720,0.263037,0.120104,0.142933,6.549042
4,0.592,0.374,0.623,0.366,90.7,2.7,0.5,13.0,8.1,0.063148,0.035920,0.027228,0.220815,0.133145,0.087670,0.365013,0.190134,0.174879,0.351025,0.176836,0.174188,5.671984


In [91]:
#Writing csv
df_full2.to_csv(r'/Users/emilyweed/Cornell2020-2021/ORIE4741/Project/ORIE-4741-Project/data/exported_data2020full.csv', index=False, header = True)
